## Fetch pool snapshots from balancer subgraph

In [15]:
# Query:
POOLS_SNAPSHOTS_QUERY = """
{{
  poolSnapshots(
    first: {first}
    skip: {skip}
    orderBy: timestamp
    orderDirection: desc
    where: {{timestamp_gte: {start_ts}, timestamp_lt: {end_ts}}}
  ) {{
    pool {{
      address
      id
      symbol
    }}
    timestamp
    protocolFee
    swapFees
    swapVolume
    liquidity
  }}
}}
"""

In [16]:
import json
from datetime import datetime
from datetime import timedelta
from typing import Dict
from typing import List
from typing import Optional

from gql import Client
from gql import gql
from gql.transport.requests import RequestsHTTPTransport

BALANCER_GRAPH_URL = "https://api.thegraph.com/subgraphs/name/balancer-labs/balancer-arbitrum-v2"

# Start date and end date are the following: start date is always previous Thursday 00:00 UTC, end date is always this Thursday 00:00 UTC:
today = datetime.today()
# Calculate days until previous Thursday (0 = Monday, 3 = Thursday)
days_until_previous_thursday = (today.weekday() - 3) % 7
# Calculate start date by subtracting days_until_previous_thursday and setting time to 00:00
start_date = today - timedelta(days=days_until_previous_thursday, hours=today.hour, minutes=today.minute,
                               seconds=today.second, microseconds=today.microsecond)
# Calculate end date by adding 7 days to the start date
end_date = start_date + timedelta(days=7)

start_ts = int(start_date.timestamp())
end_ts = int(end_date.timestamp())


# Fetch all the data from the balancer subgraph
def make_gql_client(url: str) -> Optional[Client]:
    transport = RequestsHTTPTransport(url=url, retries=3)
    return Client(
        transport=transport, fetch_schema_from_transport=True, execute_timeout=60
    )


def get_balancer_pool_snapshots() -> Optional[List[Dict]]:
    client = make_gql_client(BALANCER_GRAPH_URL)
    all_pools = []
    limit = 100
    offset = 0
    while True:
        result = client.execute(
            gql(POOLS_SNAPSHOTS_QUERY.format(first=limit, skip=offset, start_ts=start_ts, end_ts=end_ts)))
        all_pools.extend(result['poolSnapshots'])
        offset += limit
        if len(result['poolSnapshots']) < limit - 1:
            break
    return all_pools


pool_snapshots = get_balancer_pool_snapshots()

## Calculate BAL emissions per week:

In [17]:
current_year = 2023
emissions_per_week = 0
with open('../data/emissionsPerYear.json') as f:
    data = json.load(f)
for item in data['data']:
    if item['year'] == str(current_year):
        emissions_per_week = float(item['balPerWeek'])
        break
print(f'Current BAL emissions per week: {emissions_per_week}')

Current BAL emissions per week: 121929.98021178861


## Pre-process all the data in this cell

In [18]:
import os

import requests
from dotenv import load_dotenv
from web3 import Web3
from pycoingecko import CoinGeckoAPI

load_dotenv()

ARB_CHAIN_ID = 42161
BALANCER_GAUGE_URL = "https://api.thegraph.com/subgraphs/name/balancer-labs/balancer-gauges"
BALANCER_GAUGE_CONTROLLER_ADDR = Web3.toChecksumAddress("0xC128468b7Ce63eA702C1f104D55A2566b13D3ABD")
BALANCER_GAUGE_CONTROLLER_ABI = [{"stateMutability": "view", "type": "function", "name": "gauge_relative_weight",
                                  "inputs": [{"name": "addr", "type": "address"}],
                                  "outputs": [{"name": "", "type": "uint256"}]},
                                 {"stateMutability": "view", "type": "function", "name": "gauge_relative_weight",
                                  "inputs": [{"name": "addr", "type": "address"}, {"name": "time", "type": "uint256"}],
                                  "outputs": [{"name": "", "type": "uint256"}]}]

web3 = Web3(Web3.HTTPProvider(os.environ["ETHNODEURL"]))

# fetch balancer token usd price:
cg = CoinGeckoAPI()
bal_token_price = cg.get_price(ids='balancer', vs_currencies='usd')['balancer']['usd']
# Fetch all voting gauges from github json
voting_gauges_req = requests.get(
    "https://raw.githubusercontent.com/balancer/frontend-v2/master/src/data/voting-gauges.json")
if not voting_gauges_req.ok:
    raise ValueError("Failed to fetch voting gauges")
voting_gauges = voting_gauges_req.json()

# Collect arb gauges
arb_gauges = {}
for gauge in voting_gauges:
    # Only collect gauges for the arb chain and that are not killed
    if int(gauge['network']) == ARB_CHAIN_ID and gauge['isKilled'] is False:
        arb_gauges[gauge['address']] = {
            'gaugeAddress': gauge['address'],
            'pool': gauge['pool']['address'],
            'symbol': gauge['pool']['symbol'],
            'id': gauge['pool']['id'],
        }

gauge_c_contract = web3.eth.contract(address=BALANCER_GAUGE_CONTROLLER_ADDR, abi=BALANCER_GAUGE_CONTROLLER_ABI)

boost_data = {}
cap_override_data = {}
# Before, load boost data from the file
with open('../data/arbitrumGrantGuageMetadata.json') as f:
    boosties = json.load(f)
    for boost in boosties:
        boost_data[boost['gaugeAddress']] = boost.get('fixedBoost', 1)
        cap_override_data[boost['gaugeAddress']] = boost.get('capOverride', 10)
pool_protocol_fees = {}
# Collect protocol fees from the pool snapshots:
# TODO: Doesn't work properly as there can be multiple snapshots per pool per week
for gauge_addr, gauge_data in arb_gauges.items():
    for pool_snapshot in pool_snapshots:
        if Web3.toChecksumAddress(pool_snapshot['pool']['address']) == Web3.toChecksumAddress(gauge_data['pool']):
            # Since snapshots are sorted by timestamp descending, we can just take the first one we find for each pool and break
            protocol_fee = float(pool_snapshot['protocolFee']) if pool_snapshot['protocolFee'] else 0
            pool_protocol_fees[gauge_addr] = protocol_fee
            break

In [19]:
# Apply boost data to arb gauges
vote_weights = {}
combined_boost = {}
# Dynamic boost data to print out in the final table
dynamic_boosts = {}
# Collect gauge voting weights from the gauge controller on chain
for gauge_addr, gauge_data in arb_gauges.items():
    weight = gauge_c_contract.functions.gauge_relative_weight(gauge_addr).call() / 1e18 * 100
    # Calculate dynamic boost. Formula is `[Fees earned/value of bal emitted per pool + 1]`
    dollar_value_of_bal_emitted = weight * emissions_per_week * bal_token_price
    if dollar_value_of_bal_emitted != 0:
        dynamic_boost = (pool_protocol_fees.get(gauge_addr, 0) / dollar_value_of_bal_emitted) + 1
    else:
        dynamic_boost = 1
    dynamic_boosts[gauge_addr] = dynamic_boost
    # Now calculate the final boost value, which uses formula - (dynamic boost + fixed boost) - 1
    boost = (dynamic_boost + boost_data.get(gauge_addr, 1)) - 1
    combined_boost[gauge_addr] = boost
    weight *= boost
    vote_weights[gauge_addr] = weight
    arb_gauges[gauge_addr]['voteWeight'] = weight

## Calculate arbitrum distribution across gauges

In [20]:
from IPython.core.display import HTML
import pandas as pd

ARBITRUM_TO_DISTRIBUTE = 97404
VOTE_CAP_IN_PERCENT = 10  # 10% cap on any single gauge
VOTE_CAPS_IN_PERCENTS = {gauge_addr: cap_override_data.get(gauge_addr, VOTE_CAP_IN_PERCENT) for gauge_addr in
                         arb_gauges.keys()}
# Custom gauge caps taken from boost data
VOTE_CAPS = {gauge_addr: VOTE_CAPS_IN_PERCENTS[gauge_addr] / 100 * ARBITRUM_TO_DISTRIBUTE for gauge_addr in
             arb_gauges.keys()}

# Calculate total weight
total_weight = sum([gauge['voteWeight'] for gauge in arb_gauges.values()])
arb_gauge_distributions = {}
for gauge_addr, gauge_data in arb_gauges.items():
    # Calculate distribution based on vote weight and total weight
    to_distribute = ARBITRUM_TO_DISTRIBUTE * gauge_data['voteWeight'] / total_weight
    # Cap distribution
    to_distribute = to_distribute if to_distribute < VOTE_CAPS[gauge_addr] else VOTE_CAPS[gauge_addr]
    arb_gauge_distributions[gauge_addr] = {
        # 'gaugeAddress': gauge_addr,
        'pool': gauge_data['pool'],
        'symbol': gauge_data['symbol'],
        'voteWeight': gauge_data['voteWeight'],
        'distribution': to_distribute if to_distribute < VOTE_CAPS[gauge_addr] else VOTE_CAPS[gauge_addr],
        '%distribution': to_distribute / ARBITRUM_TO_DISTRIBUTE * 100,
        'boost': combined_boost.get(gauge_addr, 1),
        'staticBoost': boost_data.get(gauge_addr, 1),
        'dynamicBoost': dynamic_boosts.get(gauge_addr, 1),
        'cap': f"{cap_override_data.get(gauge_addr, VOTE_CAP_IN_PERCENT)}%"
    }


# Spend unspent arb on the gauges that are not capped yet
def recur_distribute_unspend_arb():
    unspent_arb = ARBITRUM_TO_DISTRIBUTE - sum([gauge['distribution'] for gauge in arb_gauge_distributions.values()])
    if unspent_arb > 0:
        # Find out total voting weight of uncapped gauges and mark it as 100%:
        total_uncapped_weight = sum(
            [g['voteWeight'] for g in [
                gauge for addr, gauge in arb_gauge_distributions.items() if gauge['distribution'] < VOTE_CAPS[addr]]
             ]
        )
        # Iterate over uncapped gauges and distribute unspent arb proportionally to their voting weight which is total uncapped weight
        for a, uncap_gauge in {addr: gauge for addr, gauge in arb_gauge_distributions.items() if
                               gauge['distribution'] < VOTE_CAPS[addr]}.items():
            # For each loop calculate unspend arb
            unspent_arb = ARBITRUM_TO_DISTRIBUTE - sum(
                [gauge['distribution'] for gauge in arb_gauge_distributions.values()])
            # Don't distribute more than vote cap
            distribution = min(
                uncap_gauge['distribution'] + unspent_arb * uncap_gauge['voteWeight'] / total_uncapped_weight,
                VOTE_CAPS[a])
            uncap_gauge['distribution'] = distribution
            uncap_gauge['%distribution'] = uncap_gauge['distribution'] / ARBITRUM_TO_DISTRIBUTE * 100
    # Call recursively if there is still unspent arb
    if ARBITRUM_TO_DISTRIBUTE - sum([g['distribution'] for g in arb_gauge_distributions.values()]) > 0:
        recur_distribute_unspend_arb()


recur_distribute_unspend_arb()
print(
    f"Unspent arb: {ARBITRUM_TO_DISTRIBUTE - sum([gauge['distribution'] for gauge in arb_gauge_distributions.values()])}")
print(f"Arb distributed: {sum([gauge['distribution'] for gauge in arb_gauge_distributions.values()])}")
# Remove arb gauges with 0 distribution
arb_gauge_distributions = {addr: gauge for addr, gauge in arb_gauge_distributions.items() if
                           gauge['distribution'] > 0}
arb_gauge_distributions_df = pd.DataFrame.from_dict(arb_gauge_distributions, orient='index')
arb_gauge_distributions_df = arb_gauge_distributions_df.sort_values(by='%distribution', ascending=False)
display(HTML(arb_gauge_distributions_df.to_html(index=False)))

Unspent arb: 0.0
Arb distributed: 97404.0


pool,symbol,voteWeight,distribution,%distribution,boost,staticBoost,dynamicBoost,cap
0xc6EeE8cb7643eC2F05F46d569e9eC8EF8b41b389,bb-a-USD,1.283592,17333.862378,17.795842,1.752762,1.75,1.002762,20%
0x36bf227d6BaC96e2aB1EbB5492ECec69C691943f,B-wstETH-WETH-Stable,1.063622,15869.617778,16.292573,1.957119,1.75,1.207119,20%
0xBe0f30217BE1e981aDD883848D0773A86d2d2CD4,rETH-bb-a-WETH-BPT,1.001525,13931.644102,14.302949,1.751476,1.75,1.001476,20%
0x32dF62dc3aEd2cD6224193052Ce665DC18165841,RDNT-WETH,1.684935,9740.400000,10.000000,2.725997,1.50,2.225997,10%
0xc7FA3A3527435720f0e2a4c1378335324dd4F9b3,55auraBal-45wsteth,0.791218,9740.400000,10.000000,1.000994,1.00,1.000994,10%
0xc9f52540976385A84BF416903e1Ca3983c539E34,50tBTC-50WETH,0.522396,7488.193739,7.687768,1.003142,1.00,1.003142,10%
0x45C4D1376943Ab28802B995aCfFC04903Eb5223f,wstETH-bb-a-WETH-BPT,0.483806,6589.746329,6.765375,1.755854,1.75,1.005854,20%
0x567ECfCB22205D279BB8Eed3E066989902bF03D5,DOLA/bbaUSD-BPT,0.370393,5201.107575,5.339727,1.750000,1.75,1.000000,10%
0x542F16DA0efB162D20bF4358EfA095B70A100f9E,2BTC,0.259822,3749.804581,3.849744,1.007052,1.00,1.007052,10%
0xb3028Ca124B80CFE6E9CA57B70eF2F0CCC41eBd4,50MAGIC-50USDC,0.213818,3212.010987,3.297617,1.026926,1.00,1.026926,10%


## Export to json

In [21]:
# Export to json
with open('../data/arbitrumGaugeDistribution.json', 'w') as f:
    json.dump(arb_gauge_distributions, f, indent=4)